# First part - data wrangling

In [66]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [67]:
web_page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [68]:
#import table
soup = BeautifulSoup(web_page.text, 'html.parser')
table = soup.find_all('table')[0]

#data wrangling
df = pd.read_html(str(table))[0]
postcodes = df["Postcode"]
borough = df["Borough"]
neighbourhood = df["Neighbourhood"]
df_TO= pd.DataFrame({'Postcode': postcodes, 'Borough': borough, 'Neighbourhood': neighbourhood})

#drop lines with not assigned borough
import numpy as np
df_TO.replace("Not assigned", np.nan, inplace = True)
df_TO.dropna(subset=["Borough"], axis=0, inplace=True)

#group postal areas and shape
df_TO_group=df_TO.groupby(['Postcode','Borough'])['Neighbourhood'].agg(lambda x: list(x)).reset_index()
print('shape =', df_TO_group.shape)
df_TO_group

shape = (103, 3)


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"[Rouge, Malvern]"
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"
3,M1G,Scarborough,[Woburn]
4,M1H,Scarborough,[Cedarbrae]
5,M1J,Scarborough,[Scarborough Village]
6,M1K,Scarborough,"[East Birchmount Park, Ionview, Kennedy Park]"
7,M1L,Scarborough,"[Clairlea, Golden Mile, Oakridge]"
8,M1M,Scarborough,"[Cliffcrest, Cliffside, Scarborough Village West]"
9,M1N,Scarborough,"[Birch Cliff, Cliffside West]"


# Part 2 - geolocation

In [69]:
coord= pd.read_csv('Geospatial_Coordinates.csv')
coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [70]:
df2 = pd.concat([df_TO_group, coord], axis=1)
df2.drop('Postal Code', axis = 1, inplace=True)
df2

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711
3,M1G,Scarborough,[Woburn],43.770992,-79.216917
4,M1H,Scarborough,[Cedarbrae],43.773136,-79.239476
5,M1J,Scarborough,[Scarborough Village],43.744734,-79.239476
6,M1K,Scarborough,"[East Birchmount Park, Ionview, Kennedy Park]",43.727929,-79.262029
7,M1L,Scarborough,"[Clairlea, Golden Mile, Oakridge]",43.711112,-79.284577
8,M1M,Scarborough,"[Cliffcrest, Cliffside, Scarborough Village West]",43.716316,-79.239476
9,M1N,Scarborough,"[Birch Cliff, Cliffside West]",43.692657,-79.264848


In [71]:
df_clean=df2.loc[df2['Borough'].str.contains('Toronto')]
df_clean

,Postcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,[The Beaches],43.676357,-79.293031
41,M4K,East Toronto,"[The Danforth West, Riverdale]",43.679557,-79.352188
42,M4L,East Toronto,"[The Beaches West, India Bazaar]",43.668999,-79.315572
43,M4M,East Toronto,[Studio District],43.659526,-79.340923
44,M4N,Central Toronto,[Lawrence Park],43.728020,-79.388790
45,M4P,Central Toronto,[Davisville North],43.712751,-79.390197
46,M4R,Central Toronto,[North Toronto West],43.715383,-79.405678
47,M4S,Central Toronto,[Davisville],43.704324,-79.388790
48,M4T,Central Toronto,"[Moore Park, Summerhill East]",43.689574,-79.383160
49,M4V,Central Toronto,"[Deer Park, Forest Hill SE, Rathnelly, South H...",43.686412,-79.400049


In [72]:
import folium
from geopy.geocoders import Nominatim

address = 'Toronto'
geolocator = Nominatim(user_agent="toronto")
location = geolocator.geocode(address)
Latitude = location.latitude
Longitude = location.longitude

# create map of TO using latitude and longitude values
map_TO = folium.Map(location=[Latitude, Longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_TO)  
    
map_TO